In [1]:
cd Background

/home/kunjithapathams/Desktop/Data Analytics/OCR/Background


In [3]:
import random
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import os
from PIL import Image
from skimage import io
from skimage import transform as tf
import scipy.misc
import numpy as np
import cv2


bgimage = Image.open('MasterBG.png')

In [12]:
#Create 500 random background images of size 25 * 20
for i in range(500):
    x_rand = random.randint(1,1306)
    y_rand = random.randint(1,81)
    sampleBG = bgimage.crop((x_rand,y_rand,x_rand+20,y_rand+25))
    #cd 'C:\Users\kunjithapatham.s\Desktop\Data Analytics\OCR\Background\BGList'
    filename = 'BGList/BG'+str(i)+'.png'
    #print(filename)
    sampleBG.save(filename)


In [4]:
#Write alphabets with the font type arialbd on each of the 500 generated background images
fnt = ImageFont.truetype('arialbd.ttf', 26)
alphanum = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']
for i in range(500):
    filename = 'BGList/BG'+str(i)+'.png'            
    for alpha in alphanum:
        bgimage = Image.open(filename)
        draw  = ImageDraw.Draw(bgimage)
        if alpha == 'I':
            text_pos = (4,0)
        elif alpha in ('1','2','3','4','5','6','7','8','9','0'):
            text_pos = (2,0)
        else:
            text_pos = (0,0)
        draw.text(text_pos,alpha,font=fnt)
        bgimage.save('samples/'+alpha+'_BG'+str(i)+'.png')
print('samples created....')

samples created....


In [24]:
#Skew some images (some random 300 image for each character)



shearValues = [-0.3,-0.2,-0.1,0.1,0.2,0.3]
alphanum = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']

for alpha in alphanum: 
    for i in range(100):
        randomFileNum = random.randint(0,499)
        randomFileName = 'samples/'+alpha+'_BG'+str(randomFileNum)+'.png'
        image = io.imread(randomFileName)
        afine_tf = tf.AffineTransform(shear=shearValues[random.randint(0,5)])
        modified = tf.warp(image,afine_tf)
        scipy.misc.imsave(randomFileName, modified)
print('samples updated with skewed images')
        #scipy.misc.toimage(modified, cmin=0.0, cmax=...).save(randomFileName)
        #modified.save('samples/'+alpha+'_BG'+str(randomFileNum)+'.png')
        
# Load the image as a matrix
#image = io.imread("samples/0_BG4.png")

# Create Afine transform
#afine_tf = tf.AffineTransform(shear=0.2)

# Apply transform to image data
#modified = tf.warp(image, afine_tf)

# Display the result
#io.imshow(modified)
#io.show()

samples updated with skewed images


In [5]:
def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch = image.shape   
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = image
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
                for i in image.shape]
        out[coords] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
                for i in image.shape]
        out[coords] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy

In [6]:
#Induce Noise


noise_types = ["gauss","s&p","poisson","speckle"]

#for filename in os.listdir('samples/'):
#    #print(filename)
#    i = random.randint(0,3)
#    noise_type = noise_types[i]
#    #print(noise_type)
#    image = cv2.imread('samples/'+filename,1)
#    image = noisy(noise_type,image)    
#    #print(image)
#    #gray_image = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
#    cv2.imwrite('samples/'+filename,image)
    
for filename in os.listdir('samples/'):
    #print(filename)
    #i = random.randint(0,3)
    noise_type = 'speckle' #noise_types[i]    
    image = cv2.imread('samples/'+filename,1)
    image = noisy(noise_type,image)        
    cv2.imwrite('samples/'+filename,image)

print('noice induced')
    

noice induced


In [7]:
#Magnitude Spectrum
alphanum = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']

#def applyMagnitudeSpectrum(img):
    #dft = cv2.dft(np.float32(img),flags = cv2.DFT_COMPLEX_OUTPUT)
    #dft_shift = np.fft.fftshift(dft)
    #rows, cols,value = img.shape
    #crow,ccol = rows/2 , cols/2

    # create a mask first, center square is 1, remaining all zeros
    #mask = np.zeros((rows,cols,2),np.uint8)
    #mask[crow-30:crow+30, ccol-30:ccol+30] = 1

    # apply mask and inverse DFT
    #fshift = dft_shift*mask
    #f_ishift = np.fft.ifftshift(fshift)
    #img_back = cv2.idft(f_ishift)
    #img_b#3ack = cv2.magnitude(img_back[:,:,0],img_back[:,:,1])
  
            
for alpha in alphanum: 
    for i in range(200):
        randomFileNum = random.randint(0,499)
        randomFileName = 'samples/'+alpha+'_BG'+str(randomFileNum)+'.png'
        image = io.imread(randomFileName)
        image = cv2.blur(image,(5,5))
        #image = applyMagnitudeSpectrum(image)
        cv2.imwrite('samples/'+filename,image)
        
print('images blurred')



images blurred


In [10]:
#Convert all the images to black and white using PIL
for filename in os.listdir('samples/'):
    image_file = Image.open('samples/'+filename).convert('LA')
    #image_file = image_file.convert('1')
    #image_file = image_file.convert('0')
    image_file.save('samplesBW/'+filename)
print('images converted to B&W')

images converted to B&W


In [11]:


alphanum = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']
reshaped = np.empty([0,500])
Y = np.chararray([(len(alphanum))*499,1]) 
j=0
for alpha in alphanum:
    for i in range(499):
        filename = 'samplesBW/'+alpha+'_BG'+str(i)+'.png'
        image = io.imread(filename,'0')
        reshaped = np.concatenate([reshaped,image.reshape(1,500)])
        Y[j] = alpha
        j = j+1

print('Reshaping Complete')
print('Starting to segregate Training and Validation sets')

from sklearn.model_selection import train_test_split
X_Train,X_Test,Y_Train,Y_Test = train_test_split(reshaped,Y)


print('Segregation Completed')



Reshaping Complete
Starting to segregate Training and Validation sets
Segregation Completed


In [12]:

print('starting to feature scaling')

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_Train)
X_Train = scaler.transform(X_Train)
X_Test = scaler.transform(X_Test)

print('Feature Scaling Completed')
print('Start training')
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))
mlp.fit(X_Train,Y_Train)


print('Training done...')
print('Now start predicting for the validation set')
predictions = mlp.predict(X_Test)

print('Predictions done')
print('evaluate the results')
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(Y_Test,predictions))

starting to feature scaling
Feature Scaling Completed
Start training


/home/kunjithapathams/anaconda3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:904: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training done...
Now start predicting for the validation set
Predictions done
evaluate the results
             precision    recall  f1-score   support

       b'0'       1.00      1.00      1.00       135
       b'1'       1.00      1.00      1.00       127
       b'2'       1.00      1.00      1.00       121
       b'3'       1.00      1.00      1.00       106
       b'4'       1.00      1.00      1.00       114
       b'5'       1.00      1.00      1.00       124
       b'6'       1.00      1.00      1.00       115
       b'7'       1.00      1.00      1.00       129
       b'8'       1.00      1.00      1.00       108
       b'9'       1.00      1.00      1.00       123
       b'A'       1.00      1.00      1.00       125
       b'B'       1.00      1.00      1.00       115
       b'C'       1.00      1.00      1.00       126
       b'D'       1.00      1.00      1.00       139
       b'E'       1.00      1.00      1.00       119
       b'F'       1.00      1.00      1.00       118

In [17]:

# Test with actual images from pancard
#Convert all the images to black and white
for filename in os.listdir('Test/'):
    image_file = Image.open('Test/'+filename)
    image_file = image_file.convert('LA')
    image = cv2.resize(image, (25,20)) 
    image_file.save('TestBW/'+filename)
print('test images converted to greyscale...')
    

    

test images converted to greyscale...


In [30]:
for filename in os.listdir('Test/'):
    image = io.imread('Test/'+filename)
    #image = cv2.resize(image, (25,20))
    cv2.imwrite('TestBW/'+filename,image)
    print(image.shape)
    
print('greyscale and resized images created in TestBW')
    

(21, 19, 4)
(25, 19, 4)
(26, 16, 4)
(23, 30, 4)
(29, 17, 4)
(29, 19, 4)
(28, 19, 4)
(28, 20, 4)
(27, 17, 4)
(27, 24, 4)
greyscale and resized images created in TestBW


In [19]:


pannumber = ['A','W','S','P','K','5','9','6','7','E']
reshaped1 = np.empty([0,500])
Y1=np.chararray([len(pannumber),1])
j=0
for alpha in pannumber:
    filename = 'TestBW/'+alpha+'.PNG'
    image = io.imread(filename,'0')
    image = cv2.resize(image, (25,20))
    reshaped1 = np.concatenate([reshaped1,image.reshape(1,500)])
    Y1[j] = alpha
    j = j+1

print(reshaped1.shape)
print(Y1.shape)


(10, 500)
(10, 1)


In [20]:
X1_Test = scaler.transform(reshaped1)
predictions = mlp.predict(X1_Test)
print(classification_report(Y1,predictions))

             precision    recall  f1-score   support

       b'1'       0.00      0.00      0.00         0
       b'4'       0.00      0.00      0.00         0
       b'5'       0.00      0.00      0.00         1
       b'6'       0.00      0.00      0.00         1
       b'7'       1.00      1.00      1.00         1
       b'9'       0.00      0.00      0.00         1
       b'A'       0.00      0.00      0.00         1
       b'E'       0.00      0.00      0.00         1
       b'I'       0.00      0.00      0.00         0
       b'K'       0.00      0.00      0.00         1
       b'P'       0.00      0.00      0.00         1
       b'S'       0.00      0.00      0.00         1
       b'W'       0.00      0.00      0.00         1

avg / total       0.10      0.10      0.10        10



/home/kunjithapathams/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/kunjithapathams/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [33]:
print(Y1)

[[b'A']
 [b'W']
 [b'S']
 [b'P']
 [b'K']
 [b'5']
 [b'9']
 [b'6']
 [b'7']
 [b'E']]


In [34]:
print (reshaped1)

[[ 255.  222.  224. ...,  234.  236.  204.]
 [ 253.  241.  137. ...,  127.  184.  238.]
 [ 223.  243.  166. ...,  108.  112.  255.]
 ..., 
 [ 255.  145.   78. ...,  213.  233.  210.]
 [ 217.  234.  233. ...,  199.  237.  255.]
 [ 198.  234.  176. ...,  255.  234.  198.]]
